In [1]:
import tf.records as tfr
from glob import glob
import pure_cv as vc
import json
from collections import Counter
from haskellian import iter as I
import ramda as R
import tensorflow as tf
import moveread_ocr as mo
import ocr_dataset as ods
import lines_dataset as lds
import files_dataset as fds
import fs
import os

2024-06-10 21:04:15.692944: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:04:15.848089: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 21:04:16.494581: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 21:04:18.585397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ods.len(ods.glob('data/ocr-dataset/*'))

413701

In [3]:
meta = tfr.Meta(files='*.tfrecord.gz', compression='GZIP', schema=mo.records.SCHEMA, num_samples=ods.len(ods.glob('data/ocr-dataset/*')))

In [4]:
with open('data/tfrecords/meta.json', 'w') as f:
    f.write(meta.model_dump_json(indent=2))

In [44]:
import cv2
def serialize(batch):
  img, lab = batch
  tensor = mo.parse_img(vc.decode(img, flags=cv2.IMREAD_GRAYSCALE)[..., None], already_gray=True)
  return tfr.serialize(mo.records.SCHEMA, image=tensor, label=tf.constant(lab.strip()))

for i in range(1, 12):
  print(f'\r{i}', end='', flush=True)
  ds = ods.Dataset.read(f'data/ocr-dataset/{i}')
  tfr.write(ds.samples().map(serialize), f'data/tfrecords/{i}.tfrecord.gz', compression='GZIP')

11